# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries

from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
import numpy as np
import pandas as pd
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] <urlopen error [Errno 11001] getaddrinfo failed>


In [7]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con=engine)
X = df['message']
y = df.drop(['id','message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex , text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # load stop words
    stop_words = stopwords.words("english")
    
    # lemmatize andremove stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word).strip() for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)
for col in range(36):
    print(y_test.columns[col])
    print(classification_report(y_test.iloc[:,col], y_pred[:,col]))
    print('-----------------------------------------------------')

related
             precision    recall  f1-score   support

          0       0.62      0.47      0.54      1193
          1       0.85      0.91      0.88      4010
          2       0.46      0.39      0.42        41

avg / total       0.80      0.81      0.80      5244

-----------------------------------------------------
request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      4315
          1       0.80      0.48      0.60       929

avg / total       0.88      0.89      0.87      5244

-----------------------------------------------------
offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5213
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      5244

-----------------------------------------------------
aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3036

C:\Data\Python\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {'vect__max_features': (None, 5000), 
              'clf__estimator__n_estimators': [10, 20] 
             } 


cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=1, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
from sklearn.neighbors import KNeighborsClassifier
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
     ])

In [13]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

In [14]:
y_pred = pipeline.predict(X_test)
for col in range(36):
    print(y_test.columns[col])
    print(classification_report(y_test.iloc[:,col], y_pred[:,col]))
    print('-----------------------------------------------------')

related
             precision    recall  f1-score   support

          0       0.75      0.13      0.23      1226
          1       0.79      0.99      0.88      4017

avg / total       0.78      0.79      0.72      5243

-----------------------------------------------------
request
             precision    recall  f1-score   support

          0       0.84      0.99      0.91      4329
          1       0.68      0.07      0.13       914

avg / total       0.81      0.83      0.77      5243

-----------------------------------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5227
          1       0.00      0.00      0.00        16

avg / total       0.99      1.00      1.00      5243

-----------------------------------------------------
aid_related
             precision    recall  f1-score   support

          0       0.59      0.99      0.74      3048
          1       0.70      0.04      0.07      2195

C:\Data\Python\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.